pip install rms-julian


     |████████████████████████████████| 44 kB 5.0 MB/s eta 0:00:01
  Attempting uninstall: typing-extensions
    Found existing installation: typing-extensions 4.12.2
    Uninstalling typing-extensions-4.12.2:
      Successfully uninstalled typing-extensions-4.12.2
Note: you may need to restart the kernel to use updated packages.


In [164]:
import numpy as np
import requests
import re
import math
import julian
from datetime import datetime

from datetime import datetime, timezone

from datetime import datetime, timezone
import math

from datetime import datetime, timezone
import math

def calculate_julian_date(date_string):
    """Simple one-function solution"""
    # Parse the date
    dt = datetime.strptime(date_string, "%Y-%m-%d")
    dt = dt.replace(tzinfo=timezone.utc)
    
    # Calculate Julian Date
    year = dt.year
    month = dt.month
    day = dt.day
    
    if month <= 2:
        year -= 1
        month += 12
    
    A = math.floor(year / 100)
    B = 2 - A + math.floor(A / 4)
    jd = math.floor(365.25 * (year + 4716)) + math.floor(30.6001 * (month + 1)) + day + B - 1524.5
    
    return jd

# Test it
result = calculate_julian_date("2025-12-20")
print(f"Julian Date: {result:.6f}")



# Example Usage


def load_solar_system(date_str):
    """
    Load the entire solar system (Sun + planets + Moon) into matrix_bodies format.
    
    Args:
        date_str: Date in YYYY-MM-DD format
    
    Returns:
        tuple: (matrix_bodies, body_names) or (None, None) if failed
    """
    
    # Solar System bodies with their Horizons IDs and physical properties
    SOLAR_SYSTEM = [
        {'id': '10',   'name': 'Sun',      'mass': 1.9885e30,   'radius': 696340.0},
        {'id': '199',  'name': 'Mercury',  'mass': 3.301e23,   'radius': 2440.0},
        {'id': '299',  'name': 'Venus',    'mass': 4.867e24,   'radius': 6052.0},
        {'id': '399',  'name': 'Earth',    'mass': 5.972e24,   'radius': 6371.0},
        {'id': '301',  'name': 'Moon',     'mass': 7.342e22,   'radius': 1737.0},
        {'id': '499',  'name': 'Mars',     'mass': 6.417e23,   'radius': 3390.0},
        {'id': '599',  'name': 'Jupiter',  'mass': 1.898e27,   'radius': 69911.0},
        {'id': '699',  'name': 'Saturn',   'mass': 5.683e26,   'radius': 58232.0},
        {'id': '799',  'name': 'Uranus',   'mass': 8.681e25,   'radius': 25362.0},
        {'id': '899',  'name': 'Neptune',  'mass': 1.024e26,   'radius': 24622.0},
        # Optional: Add Pluto if you want
        # {'id': '9',    'name': 'Pluto',    'mass': 1.303e22,   'radius': 1188.0},
    ]
    
    # Julian Date for the date (simplified - use the working one)
    jd = calculate_julian_date(date_str);
    
    print(f"🌌 Loading Solar System data for {date_str}...")
    print(f"   Fetching {len(SOLAR_SYSTEM)} celestial bodies from NASA")
    print("-" * 60)
    
    # Initialize matrix_bodies
    numbodies = len(SOLAR_SYSTEM)
    matrix_bodies = np.zeros((numbodies, 8))
    body_names = []
    
    successful_bodies = 0
    
    for i, body in enumerate(SOLAR_SYSTEM):
        body_id = body['id']
        body_name = body['name']
        
        print(f"{i+1:2}. Fetching {body_name:8} (ID: {body_id})...", end=' ')
        
        # NASA Horizons API call
        url = "https://ssd.jpl.nasa.gov/api/horizons.api"
        params = {
            'format': 'text',
            'COMMAND': f"'{body_id}'",
            'MAKE_EPHEM': 'YES',
            'EPHEM_TYPE': 'VECTORS',
            'CENTER': "'500@0'",
            'TLIST': str(jd),
            'TLIST_TYPE': 'JD',
            'VEC_TABLE': '2',
            'OBJ_DATA': 'NO'
        }
        
        try:
            response = requests.get(url, params=params, timeout=15)
            text = response.text
            
            if '$$SOE' not in text:
                print(f"❌ No data")
                continue
            
            # Parse the data between $$SOE and $$EOE
            data_section = text.split('$$SOE')[1].split('$$EOE')[0];
            numbers = re.findall(r'[-+]?\d*\.?\d+[Ee][-+]?\d+', data_section);
            
            if len(numbers) >= 6:
                # Extract position and velocity
                pos_x, pos_y, pos_z = 1000*float(numbers[0]), 1000*float(numbers[1]), 1000*float(numbers[2])
                vel_x, vel_y, vel_z = 1000*float(numbers[3]), 1000*float(numbers[4]), 1000*float(numbers[5])
                
                # Populate matrix_bodies row
                # Column order: [mass, radius, x, y, z, vx, vy, vz]
                matrix_bodies[i, 0] = body['mass']     # Mass (kg)
                matrix_bodies[i, 1] = 1000*body['radius']   # Radius (m)
                matrix_bodies[i, 2] = pos_x            # X position (m)
                matrix_bodies[i, 3] = pos_y            # Y position (m)
                matrix_bodies[i, 4] = pos_z            # Z position (m)
                matrix_bodies[i, 5] = vel_x            # X velocity (m/s)
                matrix_bodies[i, 6] = vel_y            # Y velocity (m/s)
                matrix_bodies[i, 7] = vel_z            # Z velocity (m/s)
                
                body_names.append(body_name)
                successful_bodies += 1
                print(f"✅")
                
                # Print a short summary
                print(f"     Position: [{pos_x:.4e}, {pos_y:.4e}, {pos_z:.4e}] m")
                print(f"     Velocity: [{vel_x:.4e}, {vel_y:.4e}, {vel_z:.4e}] m/s")
                
            else:
                print(f"❌ Parse error")
                
        except Exception as e:
            print(f"❌ Error: {str(e)[:50]}")
    
    print("-" * 60)
    
    if successful_bodies > 0:
        print(f"✅ Successfully loaded {successful_bodies} of {numbodies} bodies")
        return matrix_bodies, body_names
    else:
        print("❌ Failed to load any data")
        return None, None

def print_matrix_summary(matrix_bodies, body_names):
    """Print a summary of the loaded matrix_bodies."""
    print(f"\n{'='*70}")
    print("SOLAR SYSTEM LOADED SUCCESSFULLY!")
    print(f"{'='*70}")
    print(f"Matrix shape: {matrix_bodies.shape}")
    print(f"Number of bodies: {len(body_names)}")
    print(f"\n{'Index':<6} {'Body':<8} {'Mass(kg)':<10} {'Radius(m)':<10} {'PosX(m)':<12} {'PosY(m)':<12} {'PosZ(m)':<12} {'VelX(m/s)':<12} {'VelY(m/s)':<12} {'VelZ(m/s)':<12}")
    print("-" * 70)
    
    for i in range(len(body_names)):
        print(f"{i:<6} {body_names[i]:<8} {matrix_bodies[i,0]:<10.2e} {matrix_bodies[i,1]:<10.0f} {matrix_bodies[i,2]:<12.4e} {matrix_bodies[i,3]:<12.4e} {matrix_bodies[i,4]:<12.4e} {matrix_bodies[i,5]:<12.3e} {matrix_bodies[i,6]:<12.3e} {matrix_bodies[i,7]:<12.3e}")
    
    print(f"\nColumn format:")
    print("  0: Mass (kg)")
    print("  1: Radius (m)")
    print("  2: X position (m)")
    print("  3: Y position (m)")
    print("  4: Z position (m)")
    print("  5: X velocity (m/s)")
    print("  6: Y velocity (m/s)")
    print("  7: Z velocity (m/s)")

# ================================
# INTEGRATION WITH YOUR EXISTING CODE
# ================================


def load_planet(date_str,pid):
    """
    Load the entire solar system (Sun + planets + Moon) into matrix_bodies format.
    
    Args:
        date_str: Date in YYYY-MM-DD format
    
    Returns:
        tuple: (matrix_bodies, body_names) or (None, None) if failed
    """
    
    # Solar System bodies with their Horizons IDs and physical properties
    SOLAR_SYSTEM = [
        {'id': '10',   'name': 'Sun',      'mass': 1.9885e30,   'radius': 696340.0},
        {'id': '199',  'name': 'Mercury',  'mass': 3.301e23,   'radius': 2440.0},
        {'id': '299',  'name': 'Venus',    'mass': 4.867e24,   'radius': 6052.0},
        {'id': '399',  'name': 'Earth',    'mass': 5.972e24,   'radius': 6371.0},
        {'id': '301',  'name': 'Moon',     'mass': 7.342e22,   'radius': 1737.0},
        {'id': '499',  'name': 'Mars',     'mass': 6.417e23,   'radius': 3390.0},
        {'id': '599',  'name': 'Jupiter',  'mass': 1.898e27,   'radius': 69911.0},
        {'id': '699',  'name': 'Saturn',   'mass': 5.683e26,   'radius': 58232.0},
        {'id': '799',  'name': 'Uranus',   'mass': 8.681e25,   'radius': 25362.0},
        {'id': '899',  'name': 'Neptune',  'mass': 1.024e26,   'radius': 24622.0},
        # Optional: Add Pluto if you want
        # {'id': '9',    'name': 'Pluto',    'mass': 1.303e22,   'radius': 1188.0},
    ]
    
    # Julian Date for the date (simplified - use the working one)
    jd = calculate_julian_date(date_str);
    
    print(f"🌌 Loading Planet data for {date_str}...")
    print(f"   Fetching celestial bodies from NASA")
    print("-" * 60)
    
    # Initialize matrix_bodies
    numbodies = len(pid)
    matrix_bodies = np.zeros((numbodies, 8))
    body_names = []
    
    successful_bodies = 0
    
    lll=0;
    for i, body in enumerate(SOLAR_SYSTEM):
        body_id = body['id']
        body_name = body['name']
        if lll in pid:
            # NASA Horizons API call
            url = "https://ssd.jpl.nasa.gov/api/horizons.api"
            params = {
                'format': 'text',
                'COMMAND': f"'{body_id}'",
                'MAKE_EPHEM': 'YES',
                'EPHEM_TYPE': 'VECTORS',
                'CENTER': "'500@0'",
                'TLIST': str(jd),
                'TLIST_TYPE': 'JD',
                'VEC_TABLE': '2',
                'OBJ_DATA': 'NO'
            }


            try:
                response = requests.get(url, params=params, timeout=15);
                text = response.text;

                if '$$SOE' not in text:
                    print(f"❌ No data")
                    continue

                # Parse the data between $$SOE and $$EOE
                data_section = text.split('$$SOE')[1].split('$$EOE')[0];
                numbers = re.findall(r'[-+]?\d*\.?\d+[Ee][-+]?\d+', data_section);

                if len(numbers) >= 6:
                    # Extract position and velocity
                    pos_x, pos_y, pos_z = 1000*float(numbers[0]), 1000*float(numbers[1]), 1000*float(numbers[2])
                    vel_x, vel_y, vel_z = 1000*float(numbers[3]), 1000*float(numbers[4]), 1000*float(numbers[5])

                    # Populate matrix_bodies row
                    # Column order: [mass, radius, x, y, z, vx, vy, vz]
                    matrix_bodies[i, 0] = body['mass']     # Mass (kg)
                    matrix_bodies[i, 1] = 1000*body['radius']   # Radius (m)
                    matrix_bodies[i, 2] = pos_x            # X position (m)
                    matrix_bodies[i, 3] = pos_y            # Y position (m)
                    matrix_bodies[i, 4] = pos_z            # Z position (m)
                    matrix_bodies[i, 5] = vel_x            # X velocity (m/s)
                    matrix_bodies[i, 6] = vel_y            # Y velocity (m/s)
                    matrix_bodies[i, 7] = vel_z            # Z velocity (m/s)

                    body_names.append(body_name)
                    successful_bodies += 1
                    print(f"✅")

                    # Print a short summary
                    print(f"     Position: [{pos_x:.4e}, {pos_y:.4e}, {pos_z:.4e}] m")
                    print(f"     Velocity: [{vel_x:.4e}, {vel_y:.4e}, {vel_z:.4e}] m/s")

                else:
                    print(f"❌ Parse error")
                    
            except:
                treee=0;
                
               
        else:
            tre=0;
             
        lll=lll+1;
               
    
    print("-" * 60)
    
    if successful_bodies > 0:
        print(f"✅ Successfully loaded {successful_bodies} of {numbodies} bodies")
        return matrix_bodies, body_names
    else:
        print("❌ Failed to load any data")
        return None, None


Julian Date: 2461029.500000


In [158]:
def integrate_with_your_simulator():
    """
    Shows how to integrate the solar system loader with your existing code.
    """
    print("\n" + "="*70)
    print("ORBITAL SIMULATOR - SOLAR SYSTEM IMPORT")
    print("="*70)
    
    # User choice
    print("\nChoose an option:")
    print("1. Load entire Solar System (Sun + planets + Moon)")
    print("2. Enter bodies manually")
    
    choice = input("\nEnter choice (1-2): ").strip()
    
    if choice == '1':
        # Load entire solar system
        date_str = input("Enter date (YYYY-MM-DD, default 2025-12-15): ") or '2025-12-15'
        
        matrix_bodies, body_names = load_solar_system(date_str)
        
        if matrix_bodies is not None:
            # Print summary
            
            print_matrix_summary(matrix_bodies, body_names)
            
            # Now continue with your existing simulation code
            print(f"\n✅ Solar System data loaded into matrix_bodies!")
            print(f"   You can now proceed with your simulation using:")
            print(f"   - numbodies = {matrix_bodies.shape[0]}")
            print(f"   - matrix_bodies (ready for your calculations)")
            
            # Your existing simulation parameters
            step = float(input("\nEnter time step (seconds): "))
            endt = float(input("Enter simulation duration (seconds): "))
            Gnorm=float(input("Use standard value of G? 1 for yes,: (6.6743e-11 ) "));
            if Gnorm==1:
                G=6.6743e-11;
            else:
                G = float(input("Enter value of G:"));
            Fnorm=float(input("Use standard force law 1 for yes,: (1/r^2 ) "));
            if Fnorm==1:
                Fl=2;
            else:
                Fl = float(input("Enter alternative force law coefficient 1/r^x:"));
            
            
            print(f"\n🚀 Starting simulation with {matrix_bodies.shape[0]} bodies...")
            # Your simulation code would go here
            matrix_bodies2=matrix_bodies;
            numbodies=len(matrix_bodies);
            print_matrix_summary(matrix_bodies, body_names)
            TT=math.ceil(endt/step);
            for i in range(TT):
                matrix_bodies3=matrix_bodies2;
                Fo=np.zeros((numbodies,3));
                for j in range(numbodies):
                    for k in [x for x in range(numbodies) if x != j]:
                        dist=((matrix_bodies2[k,2]-matrix_bodies2[j,2])**2+(matrix_bodies2[k,3]-matrix_bodies2[j,3])**2+(matrix_bodies2[k,4]-matrix_bodies2[j,4])**2)**.5
                        Fs=G*matrix_bodies2[k,0]/dist**2;
                        Fd=[(matrix_bodies2[k,2]-matrix_bodies2[j,2])/dist,(matrix_bodies2[k,3]-matrix_bodies2[j,3])/dist,(matrix_bodies2[k,4]-matrix_bodies2[j,4])/dist];
                        Fd=np.asarray(Fd);
                        Fo[j,0:3]=Fo[j,0:3]+Fs*Fd;
            
                for j in range(numbodies):
                    matrix_bodies3[j,2]=matrix_bodies3[j,2]+Fo[j,0]*step**2/2+matrix_bodies3[j,5]*step;
                    matrix_bodies3[j,3]=matrix_bodies3[j,3]+Fo[j,1]*step**2/2+matrix_bodies3[j,6]*step;
                    matrix_bodies3[j,4]=matrix_bodies3[j,4]+Fo[j,2]*step**2/2+matrix_bodies3[j,7]*step;
                    matrix_bodies3[j,5]=matrix_bodies3[j,5]+Fo[j,0]*step;
                    matrix_bodies3[j,6]=matrix_bodies3[j,6]+Fo[j,1]*step;
                    matrix_bodies3[j,7]=matrix_bodies3[j,7]+Fo[j,2]*step;
        
                matrix_bodies2=matrix_bodies3;
            
            mbody=print_matrix_summary(matrix_bodies, body_names);
            
            return mbody
        
        else:
            print("Failed to load solar system. Falling back to manual input.")
            # Fall back to your original manual input
            return None, None, None
    
    elif choice == '2':
        # Your original manual input code
        print("\nManual entry selected.")
        # ... your existing manual input code here ...
        numbodies=int(input("Enter number of bodies: "));
        matrix_bodies = np.zeros((numbodies,8))
        for i in range(numbodies):
            matrix_bodies[i,0]=input("input body num " + str(i) + "mass ");
            matrix_bodies[i,1]=input("input body num " + str(i) + "radius ");
            matrix_bodies[i,2]=input("input body num " + str(i) + "x cord ");
            matrix_bodies[i,3]=input("input body num " + str(i) + "y cord ");
            matrix_bodies[i,4]=input("input body num " + str(i) + "z cord ");
            matrix_bodies[i,5]=input("input body num " + str(i) + "x velocity ");
            matrix_bodies[i,6]=input("input body num " + str(i) + "y velocity ");
            matrix_bodies[i,7]=input("input body num " + str(i) + "z velocity ");
            
        body_names=list(range(0, numbodies));
        print_matrix_summary(matrix_bodies, body_names);
        
        step = float(input("\nEnter time step (seconds): "))
        endt = float(input("Enter simulation duration (seconds): "))
        Gnorm=float(input("Use standard value of G? 1 for yes,: (6.6743e-11 ) "));
        if Gnorm==1:
            G=6.6743e-11;
        else:
            G = float(input("Enter value of G:"));
            
        Fnorm=float(input("Use standard force law 1 for yes,: (1/r^2 ) "));
        if Fnorm==1:
            Fl=2;
        else:
            Fl = float(input("Enter alternative force law coefficient 1/r^x:"));
            
        TT=math.ceil(endt/step);
        matrix_bodies2=matrix_bodies;
        for i in range(TT):
            matrix_bodies3=matrix_bodies2;
            Fo=np.zeros((numbodies,3));
            for j in range(numbodies):
                for k in [x for x in range(numbodies) if x != j]:
                    dist=((matrix_bodies2[k,2]-matrix_bodies2[j,2])**2+(matrix_bodies2[k,3]-matrix_bodies2[j,3])**2+(matrix_bodies2[k,4]-matrix_bodies2[j,4])**2)**.5
                    Fs=G*matrix_bodies2[k,0]/dist**2;
                    Fd=[(matrix_bodies2[k,2]-matrix_bodies2[j,2])/dist,(matrix_bodies2[k,3]-matrix_bodies2[j,3])/dist,(matrix_bodies2[k,4]-matrix_bodies2[j,4])/dist];
                    Fd = np.asarray(Fd)
                    Fo[j,0:3]=Fo[j,0:3]+Fs*Fd;
        
            for j in range(numbodies):
                matrix_bodies3[j,2]=matrix_bodies3[j,2]+Fo[j,0]*step**2/2+matrix_bodies3[j,5]*step;
                matrix_bodies3[j,3]=matrix_bodies3[j,3]+Fo[j,1]*step**2/2+matrix_bodies3[j,6]*step;
                matrix_bodies3[j,4]=matrix_bodies3[j,4]+Fo[j,2]*step**2/2+matrix_bodies3[j,7]*step;
                matrix_bodies3[j,5]=matrix_bodies3[j,5]+Fo[j,0]*step;
                matrix_bodies3[j,6]=matrix_bodies3[j,6]+Fo[j,1]*step;
                matrix_bodies3[j,7]=matrix_bodies3[j,7]+Fo[j,2]*step;
    
            matrix_bodies2=matrix_bodies3;
        
        
        mbody=print_matrix_summary(matrix_bodies, body_names);
        
        return mbody
# ================================
# DIRECT INTEGRATION TEMPLATE
# ================================

# Here's the minimal code to add to your existing simulator:

"""
# Add at the beginning of your code (with other imports)
import numpy as np
import requests
import re

# Add the load_solar_system() function from above

# Replace/modify your input section:

print("=== ORBITAL SIMULATOR ===")
print("Options:")
print("  1. Load real Solar System from NASA")
print("  2. Enter bodies manually")

choice = input("\nChoose option (1 or 2): ")

if choice == '1':
    # Load solar system
    date_str = input("Enter simulation date (YYYY-MM-DD) [2025-12-15]: ") or '2025-12-15'
    
    matrix_bodies, body_names = load_solar_system(date_str)
    
    if matrix_bodies is None:
        print("NASA data failed. Switching to manual input...")
        # Fall back to your original manual input
        numbodies = int(input("Enter number of bodies: "))
        matrix_bodies = np.zeros((numbodies, 8))
        # ... rest of your manual input code ...
    
else:
    # Your original manual input
    numbodies = int(input("Enter number of bodies: "))
    matrix_bodies = np.zeros((numbodies, 8))
    # ... rest of your manual input code ...

# The rest of your simulation code remains the same
print(matrix_bodies)
step = float(input("Enter time step: "))
endt = float(input("Enter duration: "))
# ... continue with your simulation ...
"""

# ================================
# TEST THE FUNCTION
# ================================

if __name__ == "__main__":
    print("Testing Solar System data loader...")
    
    # Test loading the solar system
    date_str='2025-12-23'
    matrix_bodies, body_names = load_solar_system(date_str)
    
    if matrix_bodies is not None:
        print(f"\n✅ TEST SUCCESSFUL!")
        print(f"Loaded {len(body_names)} bodies into matrix_bodies")
        
        # Show first 3 rows as example
        print(f"\nFirst 3 rows of matrix_bodies:")
        for i in range(min(3, len(body_names))):
            print(f"\n{body_names[i]}:")
            print(f"  Row {i}: {matrix_bodies[i]}")
        
        # Show how to access specific values
        print(f"\n📊 Accessing data examples:")
        print(f"  Earth mass: {matrix_bodies[3, 0]:.2e} kg (row 3, column 0)")
        print(f"  Earth X position: {matrix_bodies[3, 2]:.2e} km (row 3, column 2)")
        print(f"  Earth X velocity: {matrix_bodies[3, 5]:.2e} km/s (row 3, column 5)")
        
        # Verify the matrix structure
        print(f"\n🔍 Matrix structure verification:")
        print(f"  Shape: {matrix_bodies.shape}")
        print(f"  Number of rows (bodies): {matrix_bodies.shape[0]}")
        print(f"  Number of columns (parameters): {matrix_bodies.shape[1]}")
        print(f"  Column 0: Mass = {matrix_bodies[0, 0]:.2e} kg (Sun)")
        print(f"  Column 1: Radius = {matrix_bodies[0, 1]:.0f} km (Sun)")
        print(f"  Column 2: X position = {matrix_bodies[0, 2]:.2e} km (Sun)")
        
        # Your simulation would continue here
        print(f"\n🚀 Ready for simulation! Your matrix_bodies is populated.")

Testing Solar System data loader...
🌌 Loading Solar System data for 2025-12-23...
   Fetching 10 celestial bodies from NASA
------------------------------------------------------------
 1. Fetching Sun      (ID: 10)... ✅
     Position: [-4.6854e+08, -8.2796e+08, 1.9882e+07] m
     Velocity: [1.2465e+01, 1.6375e-01, -2.3489e-01] m/s
 2. Fetching Mercury  (ID: 199)... ✅
     Position: [-5.3471e+10, -3.9345e+10, 1.7336e+09] m
     Velocity: [1.8669e+04, -3.7246e+04, -4.7553e+03] m/s
 3. Fetching Venus    (ID: 299)... ✅
     Position: [-1.4152e+10, -1.0865e+11, -6.7193e+08] m
     Velocity: [3.4519e+04, -4.5454e+03, -2.0537e+03] m/s
 4. Fetching Earth    (ID: 399)... ✅
     Position: [-3.1376e+09, 1.4631e+11, 1.1368e+07] m
     Velocity: [-3.0265e+04, -6.5870e+02, -9.0438e-01] m/s
 5. Fetching Moon     (ID: 301)... ✅
     Position: [-2.9175e+09, 1.4598e+11, -9.6836e+06] m
     Velocity: [-2.9460e+04, -7.8963e+01, 7.0428e+01] m/s
 6. Fetching Mars     (ID: 499)... ✅
     Position: [3.1289e+

In [88]:

matrix_bodies
numbodies=10
G=6.6734E-11
Fo=np.zeros((numbodies,3));
Fl=2
j=1
k=2
matrix_bodies2=matrix_bodies

rvec=np.array([matrix_bodies2[k,2]-matrix_bodies2[j,2],matrix_bodies2[k,3]-matrix_bodies2[j,3],matrix_bodies2[k,4]-matrix_bodies2[j,4]]);
rmag=np.linalg.norm(rvec);
Fo[j,0:3]=Fo[j,0:3]+G*Fl*rvec/rmag**(Fl+1);
Fo
rvec

array([ 3.93194715e+10, -6.93070021e+10, -2.40549552e+09])

In [165]:
integrate_with_your_simulator()



ORBITAL SIMULATOR - SOLAR SYSTEM IMPORT

Choose an option:
1. Load entire Solar System (Sun + planets + Moon)
2. Enter bodies manually

Enter choice (1-2): 1
Enter date (YYYY-MM-DD, default 2025-12-15): 2025-06-25
🌌 Loading Solar System data for 2025-06-25...
   Fetching 10 celestial bodies from NASA
------------------------------------------------------------
 1. Fetching Sun      (ID: 10)... ✅
     Position: [-6.6725e+08, -8.0449e+08, 2.3595e+07] m
     Velocity: [1.2645e+01, -3.1590e+00, -2.2732e-01] m/s
 2. Fetching Mercury  (ID: 199)... ✅
     Position: [-5.9209e+10, -2.1484e+10, 3.7030e+09] m
     Velocity: [6.1492e+03, -4.3832e+04, -4.1449e+03] m/s
 3. Fetching Venus    (ID: 299)... ✅
     Position: [9.5243e+10, -5.1996e+10, -6.2137e+09] m
     Velocity: [1.6297e+04, 3.0746e+04, -5.1742e+02] m/s
 4. Fetching Earth    (ID: 399)... ✅
     Position: [8.2413e+09, -1.5260e+11, 3.2189e+07] m
     Velocity: [2.9280e+04, 1.6296e+03, -3.9727e-01] m/s
 5. Fetching Moon     (ID: 301)... ✅

In [42]:
matrix_bodies2=matrix_bodies

In [67]:
j=3;
G=6.6743E-11
numbodies=10
Fo=np.zeros((numbodies,3));
for k in [x for x in range(numbodies) if x != j]:
                        dist=((matrix_bodies2[k,2]-matrix_bodies2[j,2])**2+(matrix_bodies2[k,3]-matrix_bodies2[j,3])**2+(matrix_bodies2[k,4]-matrix_bodies2[j,4])**2)**.5
                        Fs=G*matrix_bodies2[k,0]/dist**2;
                        Fd=[(matrix_bodies2[k,2]-matrix_bodies2[j,2])/dist,(matrix_bodies2[k,3]-matrix_bodies2[j,3])/dist,(matrix_bodies2[k,4]-matrix_bodies2[j,4])/dist];
                        Fd = np.asarray(Fd);
                        Fo[j,0:3]=Fo[j,0:3]+Fs*Fd;
                        
print(Fo)
                    

[[ 0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 5.83754013e-03  1.04553322e-03 -2.67118325e-06]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00]]


In [66]:
j=3;
k=0;
G=6.6743E-11
numbodies=10
Fo=np.zeros((numbodies,3));
dist=((matrix_bodies2[k,2]-matrix_bodies2[j,2])**2+(matrix_bodies2[k,3]-matrix_bodies2[j,3])**2+(matrix_bodies2[k,4]-matrix_bodies2[j,4])**2)**.5
Fs=G*matrix_bodies2[k,0]/dist**2;
Fd=[(matrix_bodies2[k,2]-matrix_bodies2[j,2])/dist,(matrix_bodies2[k,3]-matrix_bodies2[j,3])/dist,(matrix_bodies2[k,4]-matrix_bodies2[j,4])/dist];
Fd = np.asarray(Fd)
Fo[j,0:3]=Fo[j,0:3]+Fs*Fd;
                        
print(Fo)
                    

[[ 0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 5.84601761e-03  1.07641103e-03 -1.09501979e-07]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00]]


In [63]:
Fd=[float(i) for i in Fd]

In [15]:
len(matrix_bodies)

10

In [16]:
load_solar_system('2025-12-25')


🌌 Loading Solar System data for 2025-12-25...
   Fetching 10 celestial bodies from NASA
------------------------------------------------------------
 1. Fetching Sun      (ID: 10)... ✅
     Position: [-4.6639e+08, -8.2793e+08, 1.9842e+07] m
     Velocity: [1.2456e+01, 1.9595e-01, -2.3494e-01] m/s
 2. Fetching Mercury  (ID: 199)... ✅
     Position: [-4.9886e+10, -4.5500e+10, 9.0189e+08] m
     Velocity: [2.2743e+04, -3.3940e+04, -4.8587e+03] m/s
 3. Fetching Venus    (ID: 299)... ✅
     Position: [-8.1688e+09, -1.0927e+11, -1.0256e+09] m
     Velocity: [3.4710e+04, -2.6145e+03, -2.0382e+03] m/s
 4. Fetching Earth    (ID: 399)... ✅
     Position: [-8.3644e+09, 1.4610e+11, 1.1197e+07] m
     Velocity: [-3.0223e+04, -1.7207e+03, -1.0420e+00] m/s
 5. Fetching Moon     (ID: 301)... ✅
     Position: [-8.0311e+09, 1.4590e+11, 4.1202e+06] m
     Velocity: [-2.9744e+04, -8.3598e+02, 8.6808e+01] m/s
 6. Fetching Mars     (ID: 499)... ✅
     Position: [3.5584e+10, -2.1256e+11, -5.3011e+09] m
     

(array([[ 1.98900000e+30,  6.96340000e+08, -4.66387992e+08,
         -8.27926652e+08,  1.98418542e+07,  1.24562462e+01,
          1.95947184e-01, -2.34935391e-01],
        [ 3.30100000e+23,  2.44000000e+06, -4.98861481e+10,
         -4.54999069e+10,  9.01891218e+08,  2.27430648e+04,
         -3.39399135e+04, -4.85873635e+03],
        [ 4.86700000e+24,  6.05200000e+06, -8.16876785e+09,
         -1.09270515e+11, -1.02556118e+09,  3.47099183e+04,
         -2.61449072e+03, -2.03819112e+03],
        [ 5.97200000e+24,  6.37100000e+06, -8.36437841e+09,
          1.46103231e+11,  1.11971033e+07, -3.02230880e+04,
         -1.72073338e+03, -1.04201129e+00],
        [ 7.34200000e+22,  1.73700000e+06, -8.03111757e+09,
          1.45903095e+11,  4.12017936e+06, -2.97444117e+04,
         -8.35979052e+02,  8.68082883e+01],
        [ 6.41700000e+23,  3.39000000e+06,  3.55841634e+10,
         -2.12558017e+11, -5.30113965e+09,  2.48147667e+04,
          6.15074859e+03, -4.79536478e+02],
        [ 1.8980

In [17]:
load_solar_system('2024-12-25')


🌌 Loading Solar System data for 2024-12-25...
   Fetching 10 celestial bodies from NASA
------------------------------------------------------------
 1. Fetching Sun      (ID: 10)... ✅
     Position: [-8.6477e+08, -7.3072e+08, 2.6977e+07] m
     Velocity: [1.2352e+01, -6.4993e+00, -2.0199e-01] m/s
 2. Fetching Mercury  (ID: 199)... ✅
     Position: [-5.7805e+10, 2.5670e+09, 5.5192e+09] m
     Velocity: [-1.2934e+04, -4.6547e+04, -2.6161e+03] m/s
 3. Fetching Venus    (ID: 299)... ✅
     Position: [8.2114e+10, 6.8645e+10, -3.8082e+09] m
     Velocity: [-2.2557e+04, 2.6712e+04, 1.6690e+03] m/s
 4. Fetching Earth    (ID: 399)... ✅
     Position: [-9.4217e+09, 1.4616e+11, 1.8910e+07] m
     Velocity: [-3.0216e+04, -1.8408e+03, 1.0079e+00] m/s
 5. Fetching Moon     (ID: 301)... ✅
     Position: [-9.7858e+09, 1.4598e+11, 4.4720e+06] m
     Velocity: [-2.9787e+04, -2.7049e+03, -7.6266e+01] m/s
 6. Fetching Mars     (ID: 499)... ✅
     Position: [-6.5468e+10, 2.3054e+11, 6.4577e+09] m
     Vel

(array([[ 1.98900000e+30,  6.96340000e+08, -8.64771413e+08,
         -7.30723681e+08,  2.69769397e+07,  1.23523919e+01,
         -6.49927970e+00, -2.01992055e-01],
        [ 3.30100000e+23,  2.44000000e+06, -5.78053169e+10,
          2.56699232e+09,  5.51915497e+09, -1.29340758e+04,
         -4.65470620e+04, -2.61610433e+03],
        [ 4.86700000e+24,  6.05200000e+06,  8.21141675e+10,
          6.86453899e+10, -3.80819030e+09, -2.25567551e+04,
          2.67116870e+04,  1.66899427e+03],
        [ 5.97200000e+24,  6.37100000e+06, -9.42167029e+09,
          1.46156771e+11,  1.89102850e+07, -3.02162174e+04,
         -1.84076417e+03,  1.00785934e+00],
        [ 7.34200000e+22,  1.73700000e+06, -9.78580965e+09,
          1.45981961e+11,  4.47199093e+06, -2.97872178e+04,
         -2.70490352e+03, -7.62659245e+01],
        [ 6.41700000e+23,  3.39000000e+06, -6.54684628e+10,
          2.30539467e+11,  6.45765626e+09, -2.24081169e+04,
         -4.46549114e+03,  4.56203390e+02],
        [ 1.8980

🌌 Loading Solar System data for 2024-02-10...
   Fetching 10 celestial bodies from NASA
------------------------------------------------------------
 1. Fetching Sun      (ID: 10)... ✅
     Position: [-1.16e+06, -4.76e+05, 3.12e+04] m
     Velocity: [8.92e-03, -1.17e-02, -9.16e-05] m/s
 2. Fetching Mercury  (ID: 199)... ✅
     Position: [7.87e+06, -6.83e+07, -6.34e+06] m
     Velocity: [3.85e+01, 8.92e+00, -2.80e+00] m/s
 3. Fetching Venus    (ID: 299)... ✅
     Position: [-3.79e+07, -1.03e+08, 7.46e+05] m
     Velocity: [3.27e+01, -1.20e+01, -2.05e+00] m/s
 4. Fetching Earth    (ID: 399)... ✅
     Position: [-1.15e+08, 9.36e+07, 2.69e+04] m
     Velocity: [-1.95e+01, -2.31e+01, 6.25e-04] m/s
 5. Fetching Moon     (ID: 301)... ✅
     Position: [-1.15e+08, 9.34e+07, 5.92e+02] m
     Velocity: [-1.88e+01, -2.22e+01, 5.64e-02] m/s
 6. Fetching Mars     (ID: 499)... ✅
     Position: [4.13e+07, -2.11e+08, -5.41e+06] m
     Velocity: [2.47e+01, 6.87e+00, -4.61e-01] m/s
 7. Fetching Jupiter  

(array([[ 1.98900000e+30,  6.96340000e+08, -1.16199813e+09,
         -4.76026213e+08,  3.11687700e+07,  8.91813268e+00,
         -1.16751802e+01, -9.16009784e-02],
        [ 3.30100000e+23,  2.44000000e+06,  7.87415024e+09,
         -6.82926917e+10, -6.33969462e+09,  3.85248758e+04,
          8.92082754e+03, -2.80293922e+03],
        [ 4.86700000e+24,  6.05200000e+06, -3.78726816e+10,
         -1.02628727e+11,  7.46494427e+08,  3.27295044e+04,
         -1.20058788e+04, -2.05281960e+03],
        [ 5.97200000e+24,  6.37100000e+06, -1.14883074e+11,
          9.36210862e+10,  2.69287729e+07, -1.94739573e+04,
         -2.30848279e+04,  6.25275602e-01],
        [ 7.34200000e+22,  1.73700000e+06, -1.14605175e+11,
          9.33958496e+10,  5.91890598e+05, -1.87939455e+04,
         -2.22245773e+04,  5.63783853e+01],
        [ 6.41700000e+23,  3.39000000e+06,  4.12900804e+10,
         -2.10558996e+11, -5.41287451e+09,  2.46712788e+04,
          6.86719350e+03, -4.60868807e+02],
        [ 1.8980

In [21]:
calculate_julian_date("2025-10-23")

2460971.5

TypeError: unsupported operand type(s) for +: 'datetime.datetime' and 'int'

In [53]:
b=[1,2,3]
a=np.array(b)
v=rmag=np.linalg.norm(a)
print(v)
w=a**1.9
print(w)

3.7416573867739413
[1.         3.73213197 8.06362614]


In [47]:
[2,3]**2

TypeError: unsupported operand type(s) for ** or pow(): 'list' and 'int'

In [35]:
[2,3]*[2,3]

TypeError: can't multiply sequence by non-int of type 'list'

In [ ]:
dist=((matrix_bodies2[k,2]-matrix_bodies2[j,2])**2+(matrix_bodies2[k,3]-matrix_bodies2[j,3])**2+(matrix_bodies2[k,4]-matrix_bodies2[j,4])**2)**.5
                        Fs=G*matrix_bodies2[k,0]/dist**Fl;
                        Fd=[(matrix_bodies2[k,2]-matrix_bodies2[j,2])/dist,(matrix_bodies2[k,3]-matrix_bodies2[j,3])/dist,(matrix_bodies2[k,4]-matrix_bodies2[j,4])/dist];
                        Fd = np.asarray(Fd)
                        Fo[j,0:3]=Fo[j,0:3]+Fs*Fd;
            

In [63]:

f=np.array([2,3])
f**2

array([4, 9])

In [163]:
matrix_bodies

array([[ 1.98900000e+30,  6.96340000e+08, -4.68541225e+08,
        -8.27957732e+08,  1.98824468e+07,  1.24654582e+01,
         1.63754389e-01, -2.34890107e-01],
       [ 3.30100000e+23,  2.44000000e+06, -5.34711556e+10,
        -3.93447586e+10,  1.73356456e+09,  1.86692961e+04,
        -3.72457585e+04, -4.75525182e+03],
       [ 4.86700000e+24,  6.05200000e+06, -1.41516841e+10,
        -1.08651761e+11, -6.71930955e+08,  3.45189750e+04,
        -4.54544907e+03, -2.05370136e+03],
       [ 5.97200000e+24,  6.37100000e+06, -3.13763496e+09,
         1.46308853e+11,  1.13679560e+07, -3.02650520e+04,
        -6.58698135e+02, -9.04380803e-01],
       [ 7.34200000e+22,  1.73700000e+06, -2.91748119e+09,
         1.45980314e+11, -9.68364107e+06, -2.94595336e+04,
        -7.89631595e+01,  7.04284457e+01],
       [ 6.41700000e+23,  3.39000000e+06,  3.12892643e+10,
        -2.13578554e+11, -5.21721973e+09,  2.48930039e+04,
         5.66101122e+03, -4.91717370e+02],
       [ 1.89800000e+27,  6.991100